# End-to-End Example

We learned in the [high-level](high-level.ipynb) notebook how to setup the quantum side of the calculation, which involves specifying a Hamiltonian, defining a trial wavefunction ansatz before performing shadow tomography to extract the AFQMC trial wavefunction. In this example we will repeat these steps again but now for the case of H4 which was studied in the [paper](https://arxiv.org/pdf/2106.16235.pdf). Then we will outline how to interface the output of the quantum half of the calculation with [ipie](https://github.com/JoonhoLee-Group/ipie/) to perform AFQMC with the quantum trial wavefunction.

## Define an SCF Job

We first setup a `PyscfHamiltonianParams` object which defines the SCF job we will run using [pyscf](https://github.com/pyscf/pyscf). Here we are simulation H4 in the (minimal) sto-3g basis.

In [ ]:
from recirq.qcqmc.utilities import make_output_directories
make_output_directories()

In [ ]:
from recirq.qcqmc.hamiltonian import build_hamiltonian_from_pyscf, PyscfHamiltonianParams

pyscf_params = PyscfHamiltonianParams(
    name="TEST_square_H4",
    n_orb=4,
    n_elec=4,
    geometry=(("H", (0, 0, 0)), ("H", (0, 0, 1.23)), ("H", (1.23, 0, 0)), ("H", (1.23, 0, 1.23))),
    basis="sto3g",
    multiplicity=1,
    charge=0,
    save_chkfile=True,
    overwrite_chk_file=True,
)
pyscf_hamiltonian = build_hamiltonian_from_pyscf(pyscf_params)
chk_path = pyscf_params.base_path.with_suffix(".chk")

## Build Perfect Pairing Wavefunction

Next we build a perfect pairing Hamiltonian based upon our `pyscf_params`

In [ ]:
import numpy as np
from recirq.qcqmc.trial_wf import (
    build_pp_plus_trial_wavefunction,
    PerfectPairingPlusTrialWavefunctionParams,
)

pp_params = PerfectPairingPlusTrialWavefunctionParams(
    'pp_plus_test',
    hamiltonian_params=pyscf_params,
    heuristic_layers=tuple(),
    do_pp=True,
    restricted=False,
    initial_orbital_rotation=None,
    initial_two_body_qchem_amplitudes=np.asarray([0.3, 0.4]),
    do_optimization=False,
)

trial_wf = build_pp_plus_trial_wavefunction(
    pp_params, dependencies={pyscf_params: pyscf_hamiltonian}, do_print=False
)

In [ ]:
from recirq.qcqmc.blueprint import BlueprintParamsTrialWf, build_blueprint
from recirq.qcqmc.experiment import build_experiment, SimulatedExperimentParams

blueprint_params = BlueprintParamsTrialWf(
    name='blueprint_test_medium',
    trial_wf_params=pp_params,
    n_cliffords=3,
    qubit_partition=(tuple(qubit for qubit in pp_params.qubits_jordan_wigner_ordered),),
    seed=1,
)

blueprint = build_blueprint(blueprint_params, dependencies={pp_params: trial_wf})

simulated_experiment_params = SimulatedExperimentParams(
    name='test_1',
    blueprint_params=blueprint.params,
    noise_model_name="None",
    noise_model_params=(0,),
    n_samples_per_clifford=31,
    seed=1,
)
experiment = build_experiment(
    params=simulated_experiment_params, dependencies={blueprint.params: blueprint}
)

In [ ]:
import h5py
from typing import Dict
from recirq.qcqmc.analysis import build_analysis, OverlapAnalysisParams
from recirq.qcqmc.newtilities import save_wavefunction_for_ipie
from recirq.qcqmc.utilities import Data, Params

analysis_params = OverlapAnalysisParams(
    'TEST_analysis', experiment_params=experiment.params, k_to_calculate=(1,)
)
all_dependencies: Dict[Params, Data] = {
    pyscf_params: pyscf_hamiltonian,
    pp_params: trial_wf,
    blueprint_params: blueprint,
    simulated_experiment_params: experiment,
}
analysis = build_analysis(analysis_params, dependencies=all_dependencies)
xd = save_wavefunction_for_ipie(
    hamiltonian_data=pyscf_hamiltonian, trial_wf_data=trial_wf, overlap_analysis_data=analysis
)
with h5py.File(xd.path) as fh5:
    coeffs = fh5["coeffs_rotated"][:]
    occa = fh5["occa_rotated"][:]
    occb = fh5["occb_rotated"][:]

## Run AFQMC 

Now that we have built our quantum wavefunction we can use it as a trial wavefunction in an AFQMC simulation. First we need to build a factorized Hamiltonian which is required for AFQMC. 

In particular we require the three-index Cholesky tensor (`ham.chol` below) which satisfies

$$
(pq|rs) = \sum_X L_{pq}^X L_{rs}^X.
$$

In [ ]:
from ipie.utils.from_pyscf import generate_hamiltonian_from_chk
from ipie.systems.generic import Generic

num_elec = (pyscf_params.n_elec // 2,) * 2
system = Generic(num_elec)
chk_path = pyscf_params.base_path.with_suffix(".chk")
ham = generate_hamiltonian_from_chk(chk_path)
assert ham.H1.shape == (2, pyscf_params.n_orb, pyscf_params.n_orb)
assert ham.chol.shape[0] == pyscf_params.n_orb * pyscf_params.n_orb

Next we need to build a trial wavefunction from quantum wavefunction. In practice the quantum trial is expanded as a linear combination of (orthogonal) Slater Determinants. Within ipie this style of trial wavefunction is defined as a `ParticleHole` trial wavefunction. 

In [ ]:
from ipie.trial_wavefunction.particle_hole import ParticleHole

wfn = ParticleHole((coeffs, occa, occb), num_elec, pyscf_params.n_orb, verbose=True)
wfn.half_rotate(ham)
wfn.calculate_energy(system, ham)
print(f"Trial wavefunction varitaional energy = {wfn.energy.real}")

Note that the variational energy differs slightly from the result in a previous cell. This is because the cholesky factorization uses a threshold of $1\times10^{-5}$ a stopping criteria for convergence. Reducing the parameter `chol_cut` in `generate_hamiltonian_from_chk` will yield better agreement.

At this point we are ready to run AFQMC. We need to build an `AFQMC` driver class which takes as input the factorized Hamiltonian and our `ParticleHole` trial wavefunction.

In [ ]:
from ipie.qmc.afqmc import AFQMC

qmc_driver = AFQMC.build(num_elec, ham, wfn, num_blocks=100, num_walkers=50, seed=7)

In [ ]:
qmc_driver.run(estimator_filename='./estimates.h5')

Next we can visualize the data which is by default saved to `estimates.0.h5`.

In [ ]:
from ipie.analysis.extraction import extract_observable
import matplotlib.pyplot as plt
data = extract_observable("estimates.h5")
plt.plot(data.ETotal, marker='o', label="AFQMC", color="C0")
plt.axhline(-1.969512, label="Exact Result", color="C1")
plt.legend()
plt.xlabel("Block number")
plt.ylabel("Total Energy (Ha)")

Through visual inspection we can determine when the calculation has equilibrated, after which we should be sampling the (approximate) ground state. We can use samples from this point on to estimate the AFQMC energy and error bar. The function `reblock_minimal` will perform the necessary error analysis taking into account the serial temporal correlation in the AFQMC data. 

In [ ]:
from ipie.analysis.blocking import reblock_minimal 
reblock_minimal(qmc_driver.estimators.filename)

Note that the number of walkers and the number of blocks is probably too low to obtain statistically significant results and we would advise increasing both of these. For larger scale simulations would can use MPI to distribute the work among many MPI processes. See [ipie](https://github.com/JoonhoLee-Group/ipie/) for further details.